In [75]:
import tensorflow as tf
from songe.tf_layer import tf_toolkit as tl
from songe.tf_layer import activation as ac 
from songe.toolkit import dataset_download

In [2]:
tf.enable_eager_execution()

In [40]:
ngf = 32
ndf = 64

def in_norm(x):
    return tf.contrib.layers.instance_norm(inputs = x)

def conv(x,filters,k_size,s_size,name,padding = "SAME",is_norm = True):
    net = tl.conv(x = x,
                  filters = filters, 
                  k_size = k_size, 
                  s_size = s_size, 
                  name = name ,
                  padding = padding ,
                  kernel_initializer = k_init)
    if is_norm :
        net = in_norm(net)
    return net
def conv_tran(x,filters, k_size, s_size, name , padding= "SAME",is_norm = True):
    net = tl.conv_tran(x = x ,
                       filters = filters, 
                       k_size = k_size, 
                       s_size = s_size,
                       name = name ,
                       padding=padding,
                       kernel_initializer = k_init)
    if is_norm:
        net = in_norm(net)
    return net
k_init = tf.truncated_normal_initializer(stddev = 0.02)


def build_resnet_block(x,dim,name, reuse = False):
    with tf.variable_scope("resnet_block"+name, reuse = reuse):
        net = tf.pad(x, [[0,0],[1,1],[1,1],[0,0]],"REFLECT")
        net = ac.relu(conv(net, dim,3,1,"g_resnet_block_conv"+name+str(1),padding = "VALID",kernel_initializer = k_init))
        net = tf.pad(net, [[0,0],[1,1],[1,1],[0,0]],"REFLECT")
        net = ac.relu(net + x)
    return net
    
def build_resnet_6or9_block(x,block_num,name,reuse = False):
    with tf.variable_scope(str(block_num)+name+"block_generator", reuse = reuse):
        ks = 3
        net = tf.pad(x, [[0,0],[ks,ks],[ks,ks],[0,0]], "REFLECT")
        net = ac.relu(conv(net,ngf,7,1,"g_1",padding = "VALID"))
        net = ac.relu(conv(net,ngf*2,3,2,"g_2"))
        net = ac.relu(conv(net,ngf*4,3,2,"g_3"))
            
        for i in range(block_num):
            net = resnet_blocks.build_resnet_block(net,ngf*4,"b"+str(i))
        net = ac.relu(conv_tran(net, ngf*2,3,2,"g_dc1"))
        net = ac.relu(conv_tran(net, ngf,3,2,"g_dc2"))
        net = tf.pad(net, [[0,0],[ks,ks],[ks,ks],[0,0]],"REFLECT")
        net = ac.tanh(conv(net, 3,7,1,"g_f",padding = "VALID"))
    return net 

In [43]:
def discriminator_patch(x,name , reuse = False):
    with tf.variable_scope("patch_discriminator"+ name , reuse = reuse ):
        input_ = tf.random_crop(x, [1,70,70,3])
        net = ac.lrelu(conv(input_ , ndf,4,2,"d_1"+name,is_norm = False))
        net = ac.lrelu(conv(net, ndf*2, 4,2,"d_2"+name))
        net = ac.lrelu(conv(net, ndf*4, 4,2,"d_3"+name))
        net = ac.lrelu(conv(net, ndf*8, 4,1,"d_4"+name))
        net = conv(net, 1,4,1,"d_5"+name, is_norm = False)
    return net

In [44]:
def discriminator(x,name , reuse = False):
    with tf.variable_scope("discriminator"+ name , reuse = reuse ):
        input_ = tf.random_crop(x, [1,70,70,3])
        net = ac.lrelu(conv(input_ , ndf,4,2,"d_1"+name,is_norm = False))
        net = ac.lrelu(conv(net, ndf*2, 4,2,"d_2"+name))
        net = ac.lrelu(conv(net, ndf*4, 4,2,"d_3"+name))
        net = ac.lrelu(conv(net, ndf*8, 4,2,"d_4"+name))
        net = conv(net, 1,4,1,"d_5"+name, is_norm = False)
    return net

In [ ]:
class cyclegan:
    def __init__(self, batch_size, epoch):
        self.batch_size = batch_size
        self.training_epoch = epoch 
        self.c_dim = 3
        self.img_size = [256 , 256,self.c_din]
        self.lambda = 10
        self.learning_rate = 0.0002
        self.beta1 = 0.5    
    
    def build_model(self):
        # Define placeholoder
        self.input_A = tf.placeholder(tf.float32, shape = [None]+self.img_size)
        self.input_B = tf.placeholder(tf.float32, shape = [None]+self.img_size)
        
        # making logit
        G_AtoB = build_resnet_6or9_block(input_A,9,"AtoB")
        G_BtoA = build_resnet_6or9_block(input_B,9,"BtoA")
        
        G_AtoBtoA = build_resnet_6or9_block(G_AtoB,9,"BtoA", reuse = True)
        G_BtoAtoB = build_resnet_6or9_block(G_BtoA,9,"AtoB", reuse = True)
        self.D_real_A = discriminator_patch(self.input_A,"A_dis")
        self.D_real_B = discriminator_patch(self.input_B,"B_dis")
        self.D_fake_A = discriminator_patch(G_BtoA,"A_dis",reuse = True)
        self.D_fake_B = discriminator_patch(G_AtoB,"B_dis",reuse=  True)
        
    def loss(self):
        # define loss function
        D_A_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.D_real_A, labels = tf.ones_like(self.D_real_A)))
        D_A_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.D_fake_A, labels = tf.zeros_like(self.D_fake_A)))
        
        D_B_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.D_real_B, labels = tf.ones_like(self.D_real_B)))
        D_B_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = sefl.D_fake_B, lables = tf.zeros_like(self.D_fake_B)))
        
        self.D_A_loss = (D_A_real_loss + D_A_fake_loss)*0.5
        self.D_B_loss = (D_B_real_loss + D_B_fake_loss)*0.5
        
        
        # Define G
        G_AtoB_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_A, labels = tf.ones_like(D_fake_A)))
        G_BtoA_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_B, labels = tf.ones_like(D_fake_B)))
        
        # Cycle Loss
        A_Cycle_loss = tf.reduce_mean(tf.abs(G_AtoBtoA-self.input_A))
        B_Cycle_loss = tf.reduce_mean(tf.abs(G_BtoAtoB-self.input_B))
        Cycle_loss = A_Cycle_loss + B_Cycle_loss
        
        # Total Generator Loss
        self.A_B = Cycle_loss*self.lambda + G_BtoA_loss
        self.B_A = Cycle_loss*self.lambda + G_AtoB_loss 
        
    def parameters(self):
        t_vars = tf.trainable_variables()
        self.g_vars = [var for var if 'g_' in var.name]
        self.d_vars = [var for var if 'd_' in var.name]
        
    def Optimizer(self):
        # Define optimizer function
        self.d_A_trainer = tf.train.AdamOptimizer(self.learning_rate,beta1 = self.beta1).minimize(self.D_A_loss)
        self.d_B_trainer = tf.train.AdamOptimizer(self.learning_rate,beta1 = self.beta1).minimize(self.D_B_loss)
        self.g_A_trainer = tf.train.AdamOptimizer(self.learning_rate,beta1 = self.beta1).minimize(self.A_B)
        self.g_B_trainer = tf.train.AdamOptimizer(self.learning_rate,beta1 = self.beta1),minimize(self.B_A)
    def data_preprocess(self,x):
        pass
        
    
    def train(self):
        self.build_model()
        self.loss()
        self.optimizer()
        init = tf.global_variables_initializer()
        
        with tf.Session() as sess:
            sess.run(init)
    def test(self):
        pass
    

In [73]:
import zipfile

In [74]:
download_dataset('horse2zebra')